# Final work

# Setup the learning environment

To begin, we import some library modules and functions that we will use.   


In [1]:
import cv2
import imutils
from imutils import perspective
from imutils import contours
from scipy.spatial import distance as dist
import numpy as np
import argparse
import pickle
import time
from math import sqrt
from tkinter import filedialog
from matplotlib import image
from matplotlib import pyplot as plt
from rembg import remove


In [2]:
def midpoint(ptA, ptB):
	return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)
#def mesureheight(image);
   

In [ ]:
alg = "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)

# Reading the Image
image = cv2.imread('people4.jpg')
#Take out Backgroung
image = remove(image)
image = imutils.resize(image, width = min(800, image.shape[1])) 

grayImg = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
face = haar_cascade.detectMultiScale(grayImg,1.3,4)


# getting no. of human detected
print('Human Detected : ', len(face))
Height = 0

# loop over all detected humans
for (x, y, w, h) in face:
    pad_w, pad_h = int(0.5 * w), int(0.1 * h)
    #cv2.rectangle(image, (x - pad_w, y - pad_h), (x + w + pad_w, 750 + y + h + pad_h), (0, 255, 0), 20)
    pt1 = (12,y - pad_h)
    pt2 = (12,700 + y + h - pad_h)
    color = (255, 0, 0)

    thickness=20

    
    #pt1 = (((x + pad_w) + (x + w - pad_w))/2, y + pad_h)
    #pt2 = (x + w - pad_w, y + h - pad_h) 
    cv2.line(image, pt1, pt2,color,thickness)

    # convert it to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    # perform edge detection, then perform a dilation + erosion to
    # close gaps in between object edges
    edged = cv2.Canny(gray, 5, 10)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    # find contours in the edge map
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # sort the contours from left-to-right and initialize the
    # 'pixels per metric' calibration variable
    (cnts, _) = contours.sort_contours(cnts)
    pixelsPerMetric = None
    # loop over the contours individually
    
    # show the output image
    #cv2.imshow("Image", gray)
    #cv2.imshow("Image", edged)
    #cv2.imshow("Image", image)
    #cv2.waitKey(0)
    # loop over the contours individually
    for c in cnts:
        # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(c) < 300:
            continue
        # compute the rotated bounding box of the contour
        orig = image.copy()
        box = cv2.minAreaRect(c)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        # order the points in the contour such that they appear
        # in top-left, top-right, bottom-right, and bottom-left
        # order, then draw the outline of the rotated bounding
        # box
        box = perspective.order_points(box)
        cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
        # loop over the original points and draw them
        for (x, y) in box:
            cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)
            # unpack the ordered bounding box, then compute the midpoint
            # between the top-left and top-right coordinates, followed by
            # the midpoint between bottom-left and bottom-right coordinates
            (tl, tr, br, bl) = box
            (tltrX, tltrY) = midpoint(tl, tr)
            (blbrX, blbrY) = midpoint(bl, br)
            # compute the midpoint between the top-left and top-right points,
            # followed by the midpoint between the top-righ and bottom-right
            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)
            # draw the midpoints on the image
            cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)
            # draw lines between the midpoints
            cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
                (255, 0, 255), 2)
            cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
                (255, 0, 255), 2)
            # compute the Euclidean distance between the midpoints
            dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
            dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
            # if the pixels per metric has not been initialized, then
            # compute it as the ratio of pixels to supplied metric
            # (in this case, inches)
            if pixelsPerMetric is None:
                pixelsPerMetric = dB / 0.044
                
            # compute the size of the object
            dimA = dA / pixelsPerMetric
            dimB = dB / pixelsPerMetric
            # draw the object sizes on the image
            cv2.putText(orig, "{:.2f}m".format(dimA),
                (int(tltrX - 15), int(tltrY + 10)), cv2.FONT_HERSHEY_SIMPLEX,
                0.65, (255, 255, 255), 2)
            cv2.putText(orig, "{:.2f}m".format(dimB),
                (int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
                0.65, (255, 255, 255), 2)
            # show the output image
            cv2.imshow("Image", orig)
            cv2.waitKey(0)
            
        Height = "{:.2f}m".format(dimA)
        print('Height : ', "{:.2f}m".format(dimA))
    print (Height)
    

Human Detected :  1


In [ ]:
def bmicalculculate(height):
    weight = input("How much do you weigh: ")
    bmi = weight/(height**2)
    print(bmi)

### Create model that detects humans

In [2]:
alg = "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)

#### Create a detect method

In [ ]:
def detect(frame):
    #Take out Backgroung
    frame = remove(frame)
    grayImg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face = haar_cascade.detectMultiScale(grayImg,1.3,4)    
    person = 1
    
    # loop over all detected humans
    for (x, y, w, h) in face:
        pad_w, pad_h = int(0.5 * w), int(0.1 * h)
        pt1 = (x - pad_w,y + pad_h)
        pt2 = (x - pad_w,800 + pad_h)
        color = (255, 0, 0)

        thickness=40
        cv2.line(image, pt1, pt2,color,thickness)
        cv2.putText(frame, f'person {person}', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
        person += 1
        
    cv2.putText(frame, 'Status : Detecting ', (40,40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.putText(frame, f'Total Persons : {person-1}', (40,70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.imshow('output', frame)
    return frame

### Create a method to choose the file path or human detection method

In [ ]:
def humanDetector(args):
    image_path = args["image"]
    if str(args["camera"]) == 'true' : camera = True 
    else : camera = False
    writer = None
    if args['output'] is not None and image_path is None:
        writer = cv2.VideoWriter(args['output'],cv2.VideoWriter_fourcc(*'MJPG'), 10, (600,600))
    if camera:
        print('[INFO] Opening Web Cam.')
        detectByCamera(ouput_path,writer)
    elif image_path is not None:
        print('[INFO] Opening Image from path.')
        detectByPathImage(image_path, args['output'])

### Create method to detect through camera

In [ ]:
def detectByCamera(writer):   
    video = cv2.VideoCapture(0)
    print('Detecting people...')
    while True:
        check, frame = video.read()
        frame = detect(frame)
        if writer is not None:
            writer.write(frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    video.release()
    cv2.destroyAllWindows()

### Create method to detect human from image

In [ ]:
def detectByPathImage(path, output_path):
    image = cv2.imread(path)
    #Take out Backgroung
    image = remove(image)

    image = imutils.resize(image, width = min(800, image.shape[1])) 

    result_image = detect(image)

    if output_path is not None:
        cv2.imwrite(output_path, result_image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Create a BMI calculator fundtion

In [ ]:
def bmicalculculate(height,weight):
    bmi = weight/(height**2)
    print(bmi)

### Create and argument passer function

In [ ]:
def argsParser():
    arg_parse = argparse.ArgumentParser()
    arg_parse.add_argument("-i", "--image", default=None, help="path to Image File ")
    arg_parse.add_argument("-c", "--camera", default=False, help="Set true if you want to use the camera.")
    arg_parse.add_argument("-o", "--output", type=str, help="path to optional output video file")
    args = vars(arg_parse.parse_args())

    return args

### Main Function

In [ ]:
if __name__ == "__main__":
    alg = "haarcascade_frontalface_default.xml"
    haar_cascade = cv2.CascadeClassifier(alg)

    args = argsParser()
    humanDetector(args)

# APP

In [1]:
from kivy.app import App
from kivy.uix.gridlayout import GridLayout
from kivy.uix.label import Label
from kivy.uix.image import Image
from kivy.uix.button import Button
from kivy.uix.textinput import TextInput

[INFO   ] [Logger      ] Record log in /Users/excellencesowunmi/.kivy/logs/kivy_23-03-20_0.txt
[INFO   ] [Kivy        ] v2.1.0
[INFO   ] [Kivy        ] Installed at "/Users/excellencesowunmi/opt/anaconda3/lib/python3.8/site-packages/kivy/__init__.py"
[INFO   ] [Python      ] v3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]
[INFO   ] [Python      ] Interpreter at "/Users/excellencesowunmi/opt/anaconda3/bin/python"
[INFO   ] [Logger      ] Purge log fired. Processing...
[INFO   ] [Logger      ] Purge finished!
[INFO   ] [Factory     ] 189 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_imageio, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL ES 2" graphics system
[INFO   ] [GL          ] Backend used <sdl2>
[INFO   ] [GL          ] OpenGL version <b'2.1 Metal - 83'>
[INFO   ] [GL          ] OpenGL vendor <b'Apple'>
[INFO   ] [GL          ]

In [ ]:
class BMICalculator(App):
    def build(self):
        self.window = GridLayout()
    return self.window

In [ ]:
if __name__ == "__main__":
    alg = "haarcascade_frontalface_default.xml"
    haar_cascade = cv2.CascadeClassifier(alg)

    args = argsParser()
    humanDetector(args)

In [ ]:
self.window.cols = 1

In [ ]:
self.window.add_widget(Image(source("logo_image.png")))


In [ ]:
self.ageRequest = Label(text = "Enter your year of birth...")
self.window.add_widget(self.ageRequest)

self.date = TextInput(multiline=False)
self.window.add_widget(self.date)

# Rough work

In [ ]:
import cv2

alg = "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)

# Reading the Image
image = cv2.imread('people4.jpg')
#image = imutils.resize(image, width = min(800, image.shape[1])) 

grayImg = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
face = haar_cascade.detectMultiScale(grayImg,1.3,4)


# getting no. of human detected
print('Human Detected : ', len(face))

# loop over all detected humans
for (x, y, w, h) in face:
    pad_w, pad_h = int(0.5 * w), int(0.1 * h)
#    cv2.rectangle(image, (x + pad_w, y + pad_h), (x + w - pad_w, y + h - pad_h, (0, 255, 0), 2)
    pt1 = (x + pad_w,y + pad_h)
    pt2 = (x + pad_w,800 + pad_h)
    color = (255, 0, 0)

    thickness=20

    
    #pt1 = (((x + pad_w) + (x + w - pad_w))/2, y + pad_h)
    #pt2 = (x + w - pad_w, y + h - pad_h) 
    cv2.line(image, pt1, pt2,color,thickness)


x1 = [300, 300]
y1 = [0, 900]
#x2 = [200, 400]
#y2 = [400, 400]
#plt.plot(x1, y1, x2, y2, color="white", linewidth=3)
plt.plot(x1, y1, color="white", linewidth=3)

#plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
plt.imshow(cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)[1])
plt.show()

# display the output image
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2


alg = "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)
cam = cv2.VideoCapture(0)

while True:
    _,img = cam.read()
    text="Face not detected"
    grayImg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face = haar_cascade.detectMultiScale(grayImg,1.3,4)
    for (x,y,w,h) in face:
        text="Face Detected"
        cv2.rectangle(img,(x,y),(x+w,y+h), (0,255,0),2)
    print(text)
    image = cv2.putText(img, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 2, cv2.LINE_AA)
    cv2.imshow("Face Detection", image)
    key = cv2.waitKey(10)
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

In [ ]:
def detect(frame):
    grayImg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face = haar_cascade.detectMultiScale(grayImg,1.3,4)    
    person = 1
    # loop over all detected humans
    for (x, y, w, h) in face:
        pad_w, pad_h = int(0.5 * w), int(0.1 * h)
        pt1 = (x + pad_w,y + pad_h)
        pt2 = (x + pad_w,800 + pad_h)
        color = (255, 0, 0)

        thickness=20
        cv2.line(image, pt1, pt2,color,thickness)
        cv2.putText(frame, f'person {person}', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
        person += 1
        
        
    cv2.putText(frame, 'Status : Detecting ', (40,40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.putText(frame, f'Total Persons : {person-1}', (40,70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.imshow('output', frame)
    return frame


In [ ]:
def humanDetector(args):
    image_path = args["image"]
    video_path = args['video']
    if str(args["camera"]) == 'true' : camera = True 
    else : camera = False
    writer = None
    if args['output'] is not None and image_path is None:
        writer = cv2.VideoWriter(args['output'],cv2.VideoWriter_fourcc(*'MJPG'), 10, (600,600))
    if camera:
        print('[INFO] Opening Web Cam.')
        detectByCamera(ouput_path,writer)
    elif video_path is not None:
        print('[INFO] Opening Video from path.')
        detectByPathVideo(video_path, writer)
    elif image_path is not None:
        print('[INFO] Opening Image from path.')
        detectByPathImage(image_path, args['output'])

In [ ]:
def detectByPathImage(path, output_path):
    image = cv2.imread(path)
    image = imutils.resize(image, width = min(800, image.shape[1])) 
    image = np.array(image)
    result_image = detect(image)
    if output_path is not None:
        cv2.imwrite(output_path, result_image)
        x1 = [300, 300]
        y1 = [0, 900]
        x2 = [200, 400]
        y2 = [400, 400]
        plt.plot(x1, y1, x2, y2, color="white", linewidth=3)
        
        #plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #plt.show()
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
detectByPathImage('people1.jpg', 'output_file.jpg')


### TRIED DIFFERENT HUMAN DETECTION SOFTWARE CAME OUT WITH A LOT OF ERRORS SO SETTLED FOR A FACE DETECTION SOFTWARE INSTEAD

In [ ]:
# initialize the HOG descriptor
#hog = cv2.HOGDescriptor()
#hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

HOGCV = cv2.HOGDescriptor()
HOGCV.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [ ]:
# import required libraries
import cv2

# Reading the Image
image = cv2.imread('people6.jpg')
image = imutils.resize(image, width = min(800, image.shape[1])) 

# To Grayscale
#im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#cv2.imwrite("grayscale.jpg", im)

# resizing for faster detection
frame = cv2.resize(frame, (640, 480))
# using a greyscale picture, also for faster detection
gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)


# To Black & White
#im = cv2.threshold(im, 127, 255, cv2.THRESH_BINARY)[1]
#cv2.imwrite("black-white.jpg", im)

# initialize the HOG descriptor
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())


# detect humans in input image
(humans, _) = hog.detectMultiScale(image, winStride=(4, 4),
padding=(8, 8), scale=1.03)

# getting no. of human detected
print('Human Detected : ', len(humans))

# loop over all detected humans
for (x, y, w, h) in humans:
    pad_w, pad_h = int(0.5 * w), int(0.1 * h)
#    cv2.rectangle(image, (x + pad_w, y + pad_h), (x + w - pad_w, y + h - pad_h), (0, 255, 0), 2)
    pt1 = (x + pad_w,y + pad_h)
    pt2 = (x + pad_w,800 + pad_h)
    color = (255, 0, 0)

    thickness=20

    
    #pt1 = (((x + pad_w) + (x + w - pad_w))/2, y + pad_h)
    #pt2 = (x + w - pad_w, y + h - pad_h) 
    cv2.line(image, pt1, pt2,color,thickness)


x1 = [300, 300]
y1 = [0, 900]
#x2 = [200, 400]
#y2 = [400, 400]
#plt.plot(x1, y1, x2, y2, color="white", linewidth=3)
plt.plot(x1, y1, color="white", linewidth=3)

#plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
plt.imshow(cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)[1])
plt.show()

# display the output image
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# import required libraries
import cv2

# Reading the Image
image = cv2.imread('people1.jpg')
#image = imutils.resize(image, width = min(800, image.shape[1])) 

# initialize the HOG descriptor
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())


# resizing for faster detection
frame = cv2.resize(image, (640, 480))
# using a greyscale picture, also for faster detection
gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

# detect people in the image
# returns the bounding boxes for the detected objects
boxes, weights = hog.detectMultiScale(frame, winStride=(8,8) )

boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])

# getting no. of human detected
print('Human Detected : ', len(boxes))

for (xA, yA, xB, yB) in boxes:
    # display the detected boxes in the colour picture
    cv2.rectangle(frame, (xA, yA), (xB, yB),
                      (0, 255, 0), 2)


x1 = [300, 300]
y1 = [0, 900]
#x2 = [200, 400]
#y2 = [400, 400]
#plt.plot(x1, y1, x2, y2, color="white", linewidth=3)
plt.plot(x1, y1, color="white", linewidth=3)

#plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
plt.imshow(cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)[1])
plt.show()

# display the output image
cv2.imshow("Image", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
def detect(frame):

    bounding_box_cordinates, weights =  HOGCV.detectMultiScale(frame, winStride = (4, 4), padding = (8, 8), scale = 1.03)
    
    person = 1
    for x,y,w,h in bounding_box_cordinates:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        pad_w, pad_h = int(0.15 * w) int(0.01 * h)
        #cv2.rectangle(image, (x + pad_w, y + pad_h), (x + w - pad_w, y + h - pad_h), (0, 255, 0), 2)
        cv2.putText(frame, f'person {person}', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
        person += 1
    
    cv2.putText(frame, 'Status : Detecting ', (40,40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.putText(frame, f'Total Persons : {person-1}', (40,70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.imshow('output', frame)
    return frame